- Diana Zaray Corado #191025
- Pablo Alejandro Méndez #19195
- Orlando Osberto Cabrera #19943
# Hoja de Trabajo 7 - Support Vector Machines
                                                                                                                           

In [ ]:
# Librerias
import pandas as pd
import numpy as np
import scipy.stats as sp
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


In [ ]:
# Estilos
plt.style.use('ggplot')

In [ ]:
# General functions
def calculate_frecuency(data, column, index='index', head = False):
    data_f = pd.DataFrame({
        'frecuency': data[column].value_counts(),
        'relative_frecuency (%)': data[column].value_counts(normalize=True)*100,
        'relative_acc_frecuency': data[column].value_counts(normalize=True).cumsum()
    })
    data_f.reset_index(level=[0], inplace=True)
    data_f.rename(columns={index:column}, inplace=True)
    if head:
        left_aligned_df = data_f.head(20).style.set_properties(**{'text-align': 'center'}) 
    else:
        left_aligned_df = data_f.style.set_properties(**{'text-align': 'center'})
    display(left_aligned_df)
    return data_f

In [ ]:
# General variables
cuantitative = [
    'LotFrontage',
    'LotArea',
    'MiscVal',
    'WoodDeckSF',
    'OpenPorchSF',
    'EnclosedPorch',
    '3SsnPorch',
    'ScreenPorch',
    'PoolArea',
    'GarageArea',
    'GrLivArea',
    'LowQualFinSF',
    '2ndFlrSF',
    '1stFlrSF',
    'TotalBsmtSF',
    'BsmtUnfSF',
    'BsmtFinSF2',
    'BsmtFinSF1',
    'MasVnrArea',
    'BsmtFullBath',
    'BsmtHalfBath',
    'FullBath',
    'HalfBath',
    'KitchenAbvGr',
    'TotRmsAbvGrd',
    'Fireplaces',
    'GarageCars',
    'SalePrice',
]

# Exploración de los datos
Explore los datos y explique las transformaciones que debe hacerle para generar un modelo 
de máquinas vectoriales de soporte.

In [ ]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train_shape = train.shape
test_shape = test.shape

print(f'Los datos de entrenamiento cuenta con {train_shape[0]} observaciones y {train_shape[1]} variables.\nDe igual forma, los datos de prueba tienen {test_shape[0]} observaciones y {train_shape[1]} variables')

In [ ]:
# Resumen de los datos
head = train.head().style.set_properties(**{'text-align': 'center'}) 
display(head)
del head

## Exploración de variables cualitativas
Análisis de frecuencia para observar la distribución de las variables cualitativas

#### ¿Cuál es el estilo de vivienda predominante?
Es estilo de vivienda predominante o el que más se ha vendido según los datos de entrenamiento son aquellas casas de 1 piso estilo 1946 y más pisos con diferentes estilos.

In [ ]:
subclas = calculate_frecuency(train, 'MSSubClass')
del subclas

#### ¿En qué zona se encuentran las casas más vendidas?
Las zonas en las cuales se encuentran las casas más vendidas son en residenciales con baja densidad. Algo interesante es que no existen casas en zonas industriales o de agricultura.

In [ ]:
zoning = calculate_frecuency(train, 'MSZoning')
del zoning

#### ¿Son mayores las ventas si el tipo de vía de acceso a la propiedad es pavimentado?
Sí son mayores las ventas con la vía de acceso pavimentada. De hecho, el 99.5% de las casas vendidas, en los datos de entrenamiento, tienen acceso pavimentado.

In [ ]:
street = calculate_frecuency(train, 'Street')
del street

#### Predominan las casas con todo tipo de utilidades públicas
El 99.93% de casas cuentan con todo tipo de utilidades públicas, las cuales incluyen, electricidad, gas, agua y tanque séptico.

In [ ]:
utilities = calculate_frecuency(train, 'Utilities')
del utilities

#### ¿La mayor cantidad de casas que se venden se encuentran en excelente estado?
No, las casas que más se venden son las que se encuentran en un estado promedio, aquellas que no están ni excelente pero tampoco mal. Y estás representan un 56% de las ventas totales.

In [ ]:
overall = calculate_frecuency(train, 'OverallCond')
# giving the numbers a cualitative meaning
overall['OverallCond'] = overall['OverallCond'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ['Very Excelente', 'Excelente', 'Very Good', 'Good', 'Above Average', 'Average', 'Below Average', 'Fair', 'Poor', 'Very Poor']) 

fig, ax = plt.subplots(figsize=(7, 7), subplot_kw=dict(aspect="equal"))
explode = (0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05)
wedges, texts = ax.pie(overall['relative_frecuency (%)'], wedgeprops=dict(width=0.6), startangle=-30, explode=explode)

bbox_props = dict(boxstyle="square,pad=0.5", fc="w", ec="k", lw=0.77)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/5. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(overall['OverallCond'][i] + ' ' + str(round(overall['relative_frecuency (%)'][i], 3)) + '%', xy=(x, y), xytext=(1.5*np.sign(x), 1.5*y),
                horizontalalignment=horizontalalignment, **kw)
plt.show()

#### ¿Las personas prefieren casas fundidas en concreto vertido?
Sí, al menos un 44% de las personas prefieren las casas de cemento vertido, sin embargo, un 43% prefieren las casas de bloques de cemento.

In [ ]:
foundation = calculate_frecuency(train, 'Foundation')
del foundation

#### Las casas con sótanos de al menos 100 pulgadas de altura son las más vendidas
No, las casa más vendidas son aquellas que tienen sótanos promedio, aproximadamente entre las 80-89 pulgadas.

In [ ]:
bsmt = calculate_frecuency(train, 'BsmtQual')
del bsmt

#### La mayoría de viviendas cuentan con un sistema de aire acondicionado central y calefacción en buena calidad
Al menos un 96% de las viviendas cuentan con calefacción en condiciones promedio y un 93.5% cuenta con aire acondicionado central

In [ ]:
heating = calculate_frecuency(train, 'HeatingQC')

In [ ]:
central = calculate_frecuency(train, 'CentralAir')

In [ ]:
plt.figure(figsize=(12, 6))
# Calefacción
plt.subplot(1, 2, 1)
sns.barplot(x='HeatingQC',y='frecuency',data=heating);
plt.title('Calidad de Calefacción')

# Aire acondicionado
plt.subplot(1, 2, 2)
sns.barplot(x='CentralAir',y='frecuency',data=central);
plt.title('Sistema Central de Aire Acondicionado')

plt.suptitle("Calefacción y Aire Acondicionado")

plt.show()

#### Al menos un 50% de las viviendas cuenta con garage en buenas condiciones
Un 96% de los garages de las viviendas se encuentran en condiciones promedio, y solo apenas un 0.802 tiene garage en buenas condiciones

In [ ]:
garage_cond = calculate_frecuency(train, 'GarageCond')

In [ ]:
sns.barplot(x='GarageCond', y='frecuency', data=garage_cond)
del garage_cond

#### ¿Cuáles son los precios más altos por los cuales se ha vendido una casa?
De acuerdo con los resultados obtenidos se sabe que el rango de precios en el que están las casas con los precios más altos es de 755000 hasta 485000.

In [ ]:
sale_prices  = calculate_frecuency(train, 'SalePrice', head=True)
del sale_prices

## Preprocesamiento
Como bien se sabe *Support vector machines* trabaja únicamente con datos numéricos, por lo tal, el primer paso a realizar durante la etapa de preprocesamiento es la creación de un nuevo data set que contenga únicamente las variables cuantitativas. Seguido a esto, se procede a limpiar los datos, es decir, a eliminar observaciones que tengan valores infinitos o valores faltantes, esto para que estas observaciones no afecten al momento de entrena el modelo.

In [ ]:
select_train = train[cuantitative]

### Tratamiento de valores NA

In [ ]:
# Asegurando que no existan valores nan o inf
select_train = select_train[~select_train.isin([np.nan, np.inf, -np.inf]).any(1)]

## Escalamiento de las variables
Si bien en SVM no es necesario que los datos se encuentren normalizados, algo que sí se debe de realizar es el escalamiento o estandarización de variables, esto para poder evitar que ciertas variables, con rangos numéricos muy grandes "dominen" o sean más influyentes en el modelo que aquellas variables con rangos numéricos pequeños. 

Sin embargo, el escalamiento se realizará luego de tener el conjunto de datos separados en entrenamiento y prueba, ya que al realizarlo antes se puede sesgar el modelo porque la información se puede filtrar o "mezclar" del conjunto de prueba al de entrenamiento. 

Algo interesante de SVM es que no es sensible a los outliers por lo tal no es necesario hacer un tratamiento de los mismos, ya que el SVC trabaja con un *soft margin* que permite cierto grado de *misclassification* para que los valores anormales no afecten al modelo. Si bien, en este caso se tiene un alto *bias* esto se compensa ya que la varianza del modelo es baja, permitiendo así tener predicciones constantes y evitar el *overfitting*

# Datos de entrenamiento y prueba
Use como variable respuesta la variable categórica que especifica si la casa es barata, media 
o cara

### Agregando la variable objetivo al conjunto de datos
Debido a que la variable objetivo es categórica y SVM requiere que que los datos sean representados como un vector de números es necesario codificar las categorías mediante números. Para codificar las categorías se usaran los siguientes valores:
- Baratas: 1
- Intermedias: 10
- Caras: 100

In [ ]:
# Agregando la nueva variable al data frame
conditions = [
    (select_train['SalePrice'] <= 171500),
    (select_train['SalePrice'] > 171500) & (select_train['SalePrice'] <= 295500),
    (select_train['SalePrice'] > 295500) 
    ]

values = [1, 10, 100]

select_train['HouseCategory'] = np.select(conditions, values)
del values, conditions

## Selección de variables
Para seleccionar las variables a utilizar dentro del modelo, se inició tomando en consideración únicamente las variables numéricas. Seguido a esto, se realizó una correlación entre todas las variables cuantitativas para poder analizar cuáles son las que influyen significativamente en el precio de venta y con base a esto se seleccionó el conjunto de *features* a utilizar.

In [ ]:
important_correlations = []
corr = train[cuantitative].corr(method='spearman')

for start, h1 in enumerate(cuantitative):
    for h2 in cuantitative[start + 1:]:
        if abs(corr[h1][h2]) > 0.5:
            important_correlations.append((h1, h2, corr[h1][h2]))


data = pd.DataFrame(important_correlations, columns=("Variable1", "Variable2", "Correlación"))
data_f = data.style.set_properties(**{'text-align': 'center'}) 
display(data_f)

del start, h1, h2, corr, data, data_f

Como se puede observar en la tabla y gráfica anterior se presentan aquellas variables que cuentan con una alta correlación (tomando como correlación alta a valores mayores o iguales a 0.5). A continuación se listan la correlaciones encontradas por variables:
- LotFrontage → LotArea
- GarageArea → GarageCars, SalePrice
- GrLivArea → 2ndFlrSF, FullBath, TotRmsAbvGrd, GarageCars, SalePrice
- 2ndFlrSF → HalfBath, TotRmsAbvGrd
- 1stFlrSF → TotalBsmtSF, SalePrice
- TotalBsmtSF → SalePrice
- BsmtUnfSF → BsmtFinSF1, BsmtFullBath
- FullBath → TotRmsAbvGrd, GarageCars, SalePrice
- TotRmsAbvGrd → SalePrice
- Fireplaces → SalePrice
- GargeCars → SalePrice

La estrecha correlación con la que cuentan las variables entre sí representa un potencial error para el modelo, ya que como bien se sabe, uno de los supuestos dentro del modelo de regresión logísticas es que las variables no presenten multicolinealidad ya que esto podría sesgar dicho modelo a la información "repetida" presentada por estas variables. Por lo tal, para evitar un sesgo y *overfitting* del modelo se eliminaran las variables correlacionadas, dejando solo una que represente la información de todas dentro del modelo. Las variables que se descartarán del modelo son:
- LotFrontage
- GarageCars
- TotRmsAbvGrd
- FullBath
- HalfBath
- TotalBsmtSF
- BsmtFinSF1
- BsmFullBath
- Fireplaces

In [ ]:
useless = ['LotFrontage', 
'GarageCars', 
'TotRmsAbvGrd', 
'FullBath', 
'HalfBath', 
'TotalBsmtSF', 
'BsmtFinSF1', 
'BsmtFullBath', 
'Fireplaces', 
'PoolArea', 
'LowQualFinSF', 
'BsmtFinSF2', 
'BsmtHalfBath', 
'KitchenAbvGr' ]

In [ ]:
selected_train = select_train.loc[:, ~select_train.columns.isin(useless)]

In [ ]:
# separate between target and predictors
target = selected_train.HouseCategory
predictors = selected_train.loc[:, selected_train.columns != 'HouseCategory']

In [ ]:
# stratified sample
predictors_train, predictors_test, target_train, target_test = train_test_split(
    predictors, 
    target, 
    train_size=0.7, 
    shuffle=True,
    random_state=19195
)